### Imports

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import scikitplot as skplt
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, LSTM, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPool1D, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

plt.style.use('ggplot')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/malware-analysis-datasets-api-call-sequences/dynamic_api_call_sequence_per_malware_100_0_306.csv')

In [ ]:
data.head()

In [ ]:
used_data = data.drop(['hash', 'malware'], axis=1)

In [ ]:
used_data.head()

### Classes distribution

In [ ]:
sns.countplot(data['malware'])

### splitting the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(used_data, data['malware'], test_size=0.25, 
                                                    shuffle=True, random_state=42)

In [ ]:
X_train.shape

### Building the model

In [ ]:
unique_api_calls = 307

In [ ]:
model = Sequential(name="Cnn-Lstm_model")
model.add(Embedding(input_dim=unique_api_calls, output_dim=8,
                    input_length=X_train.shape[1], name='layer_embedding'))
model.add(BatchNormalization())
model.add(Conv1D(filters = 32, kernel_size = 9, padding = 'same', activation = 'relu'))
model.add(MaxPool1D(pool_size = 2))
model.add(LSTM(units=512, return_sequences=False, dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
optimizer = Adam(lr=.0001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

### Training model

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=150, batch_size=512)

In [ ]:
model.save('behavioral-malware-detection-based-on-api-calls_model.h5')

In [ ]:
fig, ax = plt.subplots(1,2, figsize=[12,6])
ax[0].plot(history.history["loss"])
ax[0].plot(history.history["val_loss"])
ax[0].set_title(" Loss")
ax[0].legend(("Training", "validation"), loc="upper right")
ax[0].set_xlabel("Epochs")
ax[1].plot(history.history["accuracy"])
ax[1].plot(history.history["val_accuracy"])
ax[1].legend(("Training", "validation"), loc="lower right")
ax[1].set_title("Accuracy")
ax[1].set_xlabel("Epochs")

### Evaluating model

In [ ]:
y_pred = model.predict_classes(X_test)

In [ ]:
print("CNN_LSTM model classification report: \n\n {}".format(classification_report(np.array(y_test), y_pred.flatten())))

In [ ]:
probas = model.predict_proba(X_test, batch_size=64)
probas=np.array([[1-probas[i][0], probas[i][0]] for i in range(len(probas))]).reshape(probas.shape[0], 2)
skplt.metrics.plot_roc(np.array(y_test), probas, figsize=(8, 6))

In [ ]:
skplt.metrics.plot_precision_recall(np.array(y_test), probas, figsize=(8, 6))

In [ ]:
ax=skplt.metrics.plot_confusion_matrix(y_test, y_pred, figsize=(8, 7))
tickx=ax.set_xticklabels(['Benign', 'Malware'])
ticky=ax.set_yticklabels(['Benign', 'Malware'])